# Dynamic Programming
## Longest Common Subsequence

A sequence is a subset of elements of a string if any number of elements from that string are removed, but the original order is preserved. For example, 'abc' has the substrings: {'a', 'b', 'c', 'ab', 'bc', 'ac', 'abc'}. Notice 'ca' is NOT a subsequence because 'a' and 'c' do not appear in the order that they appear in the original string.

The longest common subsequence algorithm takes two strings as input and outputs the longest subsequence that belongs to both strings.

Given string1 and string2 lengths n and m, use the following method:

1. Create a n+1 x m+1 matrix that has the characters of string1 as rows and the characters of string2 as columns with a column of padding on the far left and at the top.
2. Initialize all values to zeros.
3. Starting at (2, 2) if you are indexed by 1, compare the row and column characters. If they are the same, add 1 to the value in (i-1, j-1). If they are not, take the max value from the call one to the left left or one above: max((i-1, j), (i, j-1)).

Note: If there are multiple options, then it means the solution is non-unique.

The tables below demonstrate the progress of the algorithm.

In [7]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

def style_specific_cell(df, row, col, c):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    df_styler.iloc[row, col] = 'color: black;background-color: ' + c
    return df_styler

def style_lcs_match(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    #upper left
    df_styler.iloc[row-1, col-1] = 'color: black; background-color: lightgreen'
    #row char
    df_styler.iloc[row, 0] = 'color: black; background-color: yellowgreen; font-weight:bold'
    #column char
    df_styler.iloc[0, col] = 'color: black; background-color: yellowgreen; font-weight:bold'
    return df_styler

def style_lcs_no_match(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    #row char
    df_styler.iloc[row, 0] = 'color: black; background-color: lightcoral; font-weight:bold'
    #column char
    df_styler.iloc[0, col] = 'color: black; background-color: lightcoral; font-weight:bold'
    #top
    df_styler.iloc[row-1, col] = 'color: black; background-color: lightgreen'
    #left
    df_styler.iloc[row, col-1] = 'color: black; background-color: lightgreen'
    return df_styler

def style_lcs_final(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)

    v = df.iloc[row, col]
    while(v > 0):
        df_styler.iloc[row, col] = 'color: black; background-color: gold; font-weight:bold'
        if(df.iloc[row, col-1] == df.iloc[row, col]):
            col -=1
        elif(df.iloc[row-1, col] == df.iloc[row, col]):
            row -=1
        else:
            df_styler.iloc[row, 0] = 'color: black; background-color: yellowgreen; font-weight:bold'
            df_styler.iloc[0, col] = 'color: black; background-color: yellowgreen; font-weight:bold'
            col -=1
            row -=1
            
        v = df.iloc[row, col]
    return df_styler

string1 = '--amdfb'
string2 = '--adaba'

df = pd.DataFrame(np.zeros((len(string2)-1, len(string1)))).astype(int)
df.loc[-1] = [x for x in string1]
df.index = df.index + 1
df = df.sort_index()
df.iloc[:,0] = [x for x in string2]

df.style.hide(axis=1).hide(axis=0)

-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,0,0,0,0,0
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


In [8]:
i = 2
for row_char in df.iloc[2:, 0]:
    j = 2
    for column_char in df.iloc[0, 2:]:
        if row_char == column_char:
            df.iloc[i, j] = df.iloc[i-1, j-1] + 1
            temp = df.style.apply(style_lcs_match, row = i, col = j, axis = None).hide(axis=1).hide(axis=0)
        else:
            df.iloc[i, j] = max(df.iloc[i-1, j], df.iloc[i, j-1])
            temp = df.style.apply(style_lcs_no_match, row = i, col = j, axis = None).hide(axis=1).hide(axis=0)
            

        display(temp)

        j += 1

    i += 1

max_value = [df.iloc[df.shape[0]-1, df.shape[1]-1], df.shape[0]-1, df.shape[1]-1]

-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,0,0,0,0
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,0,0,0
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,0,0
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,0
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,0,0,0,0,0
a,0,0,0,0,0,0
b,0,0,0,0,0,0
a,0,0,0,0,0,0


...

-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,0,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,1,0,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,1,2,0,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,1,2,2,0


-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,1,2,2,3


In [9]:
df.style.apply(style_lcs_final, row = max_value[1], col = max_value[2], axis = None).hide(axis=1).hide(axis=0)

-,-,a,m,d,f,b
-,0,0,0,0,0,0
a,0,1,1,1,1,1
d,0,1,1,2,2,2
a,0,1,1,2,2,2
b,0,1,1,2,2,3
a,0,1,1,2,2,3


The final table above shows the method of obtaining the final answer:
1. Find the last cell of the table.
2. If the value to the left or top is equivalent to the current value, move to that value, but do not select any characters.
3. If step 2 doesn't apply, move to the value to the top left corner (i-1, j-1) and select the corresponding character of the value you are moving from.

If there is a choice between left, top, and diagonal, it means there is more than one correct answer. This implementation prioritizes in this order: go left if possible, go up if possible, go diagonal.

The final solution for this problem is: 'adb'

In [ ]:
#specific column 
col_loc_add = df.columns.get_loc('a') + 2
col_loc_drop = df.columns.get_loc('b') + 2

df.style.apply(style_specific_cell, row = 0, col = 0, axis = None, c = 'red')\
    .set_table_styles(
     [{'selector': f'th:nth-child({col_loc_add})',
       'props': 'background-color: #67c5a4'},
      {'selector': f'th:nth-child({col_loc_drop})',
       'props': 'background-color: #ff9090'}])

#specific row
df.style.set_table_styles(
        {"b": [{'selector': 'th', 'props': 'background-color: green'}], 
        "m": [{'selector': 'th', 'props': 'background-color: blue'}]}, axis=1)

#they don't work well together